In [3]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from official.nlp import optimization 

import matplotlib.pyplot as plt

import shutil

ImportError: dlopen(/opt/miniconda3/envs/alyra_env/lib/python3.8/site-packages/tensorflow_text/core/pybinds/tflite_registrar.so, 0x0002): Symbol not found: __ZN4absl12lts_2023012516raw_log_internal21internal_log_functionE
  Referenced from: <C0B18DF4-0646-35AC-A9EB-25706D456E2F> /opt/miniconda3/envs/alyra_env/lib/python3.8/site-packages/tensorflow_text/core/pybinds/tflite_registrar.so
  Expected in:     <5E7D9A1A-DE84-3B32-93CD-B46FF6AB6926> /opt/miniconda3/envs/alyra_env/lib/python3.8/site-packages/tensorflow/libtensorflow_framework.2.dylib

In [ ]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')


In [ ]:
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
train_dir = os.path.join(dataset_dir, 'train')

2025-04-01 16:42:12.857373: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

Epoch 1/5
1875/1875 [==============================] - 46s 22ms/step - loss: 0.2953 - accuracy: 0.9136 - val_loss: 0.1449 - val_accuracy: 0.9560
Epoch 2/5
1875/1875 [==============================] - 30s 16ms/step - loss: 0.1450 - accuracy: 0.9568 - val_loss: 0.1014 - val_accuracy: 0.9707
Epoch 3/5
1875/1875 [==============================] - 14s 8ms/step - loss: 0.1077 - accuracy: 0.9681 - val_loss: 0.0873 - val_accuracy: 0.9750
Epoch 4/5
1875/1875 [==============================] - 12s 7ms/step - loss: 0.0899 - accuracy: 0.9727 - val_loss: 0.0764 - val_accuracy: 0.9756
Epoch 5/5
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0769 - accuracy: 0.9756 - val_loss: 0.0813 - val_accuracy: 0.9763


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

batch_size = 32 
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [ ]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [ ]:
text_test = ['i love that movie!!']
text_preprocessed = bert_preprocess_model(text_test)

print(list(text_preprocessed.keys()))

In [ ]:
bert_results = bert_model(text_preprocessed)

In [ ]:
print(bert_results)

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')

  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')

  outputs = encoder(encoder_inputs)

  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.2)(net)
  net = tf.keras.layers.Dense(1, name='classifier')(net)

  return tf.keras.Model(text_input, net)


In [ ]:
classifier_model = build_classifier_model()
raw_result = classifier_model(tf.constant(text_test))

print(tf.sigmoid(raw_result))

In [ ]:
classifier_model.summary()

In [ ]:
tf.keras.utils.plot_model(classifier_model)

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [ ]:
epochs = 5

steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')


In [ ]:
classifier_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
history = classifier_model.fit(x=train_ds, validation_data=val_ds, epochs=epochs)

In [ ]:
loss, accuracy = classifier_model.evaluate(test_ds)

In [ ]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i][0]:.6f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


In [ ]:
examples = [
    'i love that movie!!',
    'thas movie was okish',
    'i realy think that movie was bad',
    'i loved but was awful',
    'ok tier',
    'was awful but i loved',
    'i loved but i was too afraid'
]

results = tf.sigmoid(classifier_model(tf.constant(examples))) 

In [ ]:
print_my_examples(examples, results)